# Imports

In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
import seaborn as sns
import numpy as np
import datetime as dt
from collections import Counter
from dotenv import load_dotenv, find_dotenv
from urllib.parse import urljoin
import requests
from io import StringIO, BytesIO

# Data

In [2]:
bike_df_all = pd.read_csv('raw_data/bike_loans.csv', sep=';').drop(columns=['Importe', 
                                                                        'DescripcionImporte', 
                                                                        'operario'])
bike_df_all['Fecha_Prestamo'] = bike_df_all['Fecha_Prestamo'].astype('datetime64[ns]')
bike_df_all['Fecha_Devolucion'] = bike_df_all['Fecha_Devolucion'].astype('datetime64[ns]')

bike_df_all.dtypes
bike_df_all = bike_df_all.dropna()
bike_df = bike_df_all[bike_df_all.Fecha_Devolucion <= '31/12/2019']
bike_df.shape

(10737486, 10)

In [3]:
bike_df.dtypes

Id_Historico_Prestamo               int64
Id_Usuario                          int64
Id_Tag_Bicicleta                   object
Fecha_Prestamo             datetime64[ns]
Fecha_Devolucion           datetime64[ns]
Id_Aparcamiento_Origen              int64
Posicion_Origen                     int64
Id_Aparcamiento_Destino           float64
Posicion_Destino                  float64
Num_Bici_Hist                     float64
dtype: object

# Weather

In [30]:
# API KEY
load_dotenv(find_dotenv())
OWM_API = os.environ.get("OWM_API")

In [70]:
bike_df.Fecha_Prestamo.min().value

1420052442000000000

In [29]:
req = f'http://history.openweathermap.org/data/2.5/history/city?q=Munchen,DE&appid={OWM_API}'
r = requests.get(req)
r.json()

{'code': 404000, 'message': 'no data'}

In [71]:
# req = f'http://history.openweathermap.org/data/2.5/history/city?id=2885679&type=hour&appid={OWM_API}'
# r = requests.get(req)
# r.content

In [72]:
# r.json()

In [94]:
bike_df.Fecha_Prestamo.min()

Timestamp('2014-12-31 19:00:42')

In [92]:
req = 'http://history.openweathermap.org/data/2.5/history/city'
start = bike_df.Fecha_Prestamo.min().value
end = bike_df.Fecha_Devolucion.max().value
params = {
    'id':'3674962', # ID of medellin
    'type':'hour',
    'start':str(start)[:10], # unix time
    'end':str(end)[:10],
    'appid': OWM_API
}

r = requests.get(req, params=params)
r.status_code

400

In [93]:
r.json()

{'code': 400000, 'message': 'data is available only in the last year'}